In [50]:
import numpy as np 
from scipy.stats import norm

In [91]:
def sigmaG(x):
    # robust replacement for standard deviation
    return 0.7413*(np.quantile(x, .75, interpolation='midpoint')-np.quantile(x, .25, interpolation='midpoint'))

def getChi2(x, sig, m, Nparam):
    # given data x, uncertainty sig, model m, and the number of parameters
    # for that model, compute: 
    # 1) chi2dof: standard chi2 per degree of freedom, using st.dev. of (x-m)/sig
    # 2) chi2dofR: robust chi2 per degree of freedom (using interquartile range)
    # 3) meddev: median deviation of abs((x-m)/sig) 
    # 4) maxdev: maximum deviation of abs((x-m)/sig)  
    # and return (chi2dof, chi2dofR, meddev, maxdev) 
    z = (x-m)/sig
    Ndata = len(z)
    chi2  = (Ndata-1) * np.std(z)  # using st.dev for symmetry with chi2R   
    chi2R = (Ndata-1) * sigmaG(z)  # but could also use chi2 = sum(z^2) instead
    dof = Ndata - Nparam
    return chi2/dof, chi2R/dof, np.median(np.abs(z)), np.max(np.abs(z)) 

In [84]:
# generate a sample for testing (1000 points drawn from a normal distribution)
model=111
error=33
sampleSize = 1000
x = norm(model, error).rvs(sampleSize) 

In [85]:
m = 0*x + np.mean(x)   # model is simply the sample mean (and 0*x is a trick to get the same vector length)
sig = 0*x + error      # homoscedastic errors 
Nparam = 1             # because model has only one parameter 
chi2dof, chi2dofR, meddev, maxdev = getChi2(x, sig, m, Nparam)

In [86]:
print(chi2dof, chi2dofR, meddev, maxdev)

0.9545050622558837 0.9376031444701913 0.638426274325022 2.8434860714298154


In [87]:
# now add 100 outliers with 10 times larger errors
xBad = norm(model, 10*error).rvs(100)
xAll = np.concatenate([x,xBad])
mAll = 0*xAll + model
# note that all points, including outliers, have the same (smaller) uncertainty
# for this reason, we expect chi2dof to be larger than 1, but chi2R to stay ~1 
sigAll = 0*xAll + error

In [88]:
chi2dof, chi2dofR, meddev, maxdev = getChi2(xAll, sigAll, mAll, Nparam)

In [89]:
print(chi2dof, chi2dofR, meddev, maxdev)

3.090250182896888 1.0341662342066733 0.703316286610676 26.05435286740126


In [90]:
# as expected, 10% of outliers significantly increased chi2dof